## A.I. Assignment 5

## Learning Goals

By the end of this lab, you should be able to:
* Get more familiar with tensors in pytorch 
* Create a simple multilayer perceptron model with pytorch
* Visualise the parameters


### Task

Build a fully connected feed forward network that adds two bits. Determine the a propper achitecture for this network (what database you use for this problem? how many layers? how many neurons on each layer? what is the activation function? what is the loss function? etc)

Create at least 3 such networks and compare their performance (how accurate they are?, how farst they are trained to get at 1 accuracy?)

Display for the best one the weights for each layer.


In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
import pandas as pd



In [71]:
data_in = torch.tensor([[0., 0.], # A and B elements
                        [0., 1.],
                        [1., 0.],
                        [1., 1.]])

#Perform A XOR B :=> SUM
#Perform A AND B :=> CARRY

data_target = torch.tensor([[0., 0.],  # s=0, c=0
                            [1., 0.],  # s=1, c=0
                            [1., 0.],  # s=1, c=0
                            [0., 1.]]) # s=0, c=1


In [72]:
def train_model(model, lr=0.1, epochs=10000):
    
    criterion = nn.MSELoss()
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    losses = []
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        output = model(data_in)
        loss = criterion(output, data_target)
        
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        
        if loss.item() < 1e-4:
            break
            
    return model, losses

In [73]:
model1 = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(2, 4)),
    ('act1', nn.Sigmoid()),
    ('fc2', nn.Linear(4, 2)),
    ('act2', nn.Sigmoid())
]))

model2 = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(2, 3)),
    ('act1', nn.ReLU()),
    ('fc2', nn.Linear(3, 3)),
    ('act2', nn.ReLU()),
    ('fc3', nn.Linear(3, 2)),
    ('act3', nn.Sigmoid())
]))

model3 = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(2, 6)),
    ('act1', nn.Tanh()),
    ('fc2', nn.Linear(6, 2)),
    ('act2', nn.Sigmoid())
]))

In [74]:
model1, loss1 = train_model(model1)
model2, loss2 = train_model(model2)
model3, loss3 = train_model(model3)

In [69]:
df = pd.DataFrame({
    "Input": [list(x.numpy()) for x in data_in],
    "Target": [list(y.numpy()) for y in data_target],
    "Model1": [list(o.detach().numpy()) for o in model1(data_in)],
    "Model2": [list(o.detach().numpy()) for o in model2(data_in)],
    "Model3": [list(o.detach().numpy()) for o in model3(data_in)],
})

print("\nResult Compare")
print(df)

final_losses = [loss1[-1], loss2[-1], loss3[-1]]
best_model_index = final_losses.index(min(final_losses))
best_model = [model1, model2, model3][best_model_index]

print(f"\nBest model: model{best_model_index + 1}")
print(f"Loss final: {final_losses[best_model_index]:.6f}")

for name, param in best_model.named_parameters():
    print(f"{name}:\n{param.data}\n")


Result Compare
        Input      Target                        Model1              Model2  \
0  [0.0, 0.0]  [0.0, 0.0]  [0.011490224, 0.00044933855]  [0.50005007, 0.25]   
1  [0.0, 1.0]  [1.0, 0.0]     [0.99029225, 0.006001152]  [0.50005007, 0.25]   
2  [1.0, 0.0]  [1.0, 0.0]      [0.99050736, 0.00598411]  [0.50005007, 0.25]   
3  [1.0, 1.0]  [0.0, 1.0]      [0.012343684, 0.9839686]  [0.50005007, 0.25]   

                         Model3  
0  [0.014913199, 7.7461145e-06]  
1     [0.98986393, 0.002876999]  
2    [0.98993087, 0.0028583778]  
3     [0.015353615, 0.98953885]  

Best model: model3
Loss final: 0.000099
fc1.weight:
tensor([[-1.5975, -1.5931],
        [ 1.8304,  1.8635],
        [-3.2628, -3.1454],
        [ 0.4688,  0.6411],
        [ 4.1205,  3.9494],
        [ 1.9898,  1.9442]])

fc1.bias:
tensor([ 1.9743, -2.4173,  1.6054, -2.0983, -2.0570, -2.6564])

fc2.weight:
tensor([[ 1.5202, -2.0871, -2.8967,  1.3549,  3.3668, -3.1832],
        [-2.1998,  2.6895, -1.3760,  1.6242, 